<a href="https://colab.research.google.com/github/jihyoung-lee/stock_news_classifier/blob/main/news_title_data_set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install beautifulsoup4 requests pandas --quiet

In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# 검색할 키워드와 목표 수량 설정 + 라벨 정의
keywords = {
    "주가+상승": 1,
    "주가+급등": 1,
    "주가+하락": 0,
    "주가+폭락": 0
}
target_per_keyword = 200

# ✅ 키워드 띄어쓰기 변환 딕셔너리
keyword_spacing = {
    '주가상승': '주가 상승',
    '주가급등': '주가 급등',
    '주가하락': '주가 하락',
    '주가폭락': '주가 폭락'
}

# 결과 저장 리스트
news_titles = []
news_keywords = []
news_labels = []

# 키워드별 수집
for keyword, label in keywords.items():
    print(f"'{keyword}' 뉴스 검색 중...")

    page = 1
    collected = 0

    while collected < target_per_keyword:
        url = f"https://search.daum.net/search?nil_suggest=btn&w=news&DA=SBC&cluster=y&q={keyword}&p={page}"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # 뉴스 제목 추출
        titles = soup.select('div.c-item-content strong.tit-g.clamp-g a')

        if not titles:
            print(f"❗ '{keyword}' 관련 뉴스가 더 이상 없습니다.")
            break

        for tag in titles:
            title = tag.get_text(strip=True)

            # ✅ 키워드 띄어쓰기 변환
            for wrong, correct in keyword_spacing.items():
                title = title.replace(wrong, correct)

            title = title.replace("'", "").replace('"', '')  # 따옴표 제거
            title = ' '.join(title.split())  # 중복 띄어쓰기 정리

            if title not in news_titles and title != '':
                news_titles.append(title)
                news_keywords.append(keyword)
                news_labels.append(label)
                collected += 1

            if collected >= target_per_keyword:
                break

        page += 1
        time.sleep(0.5)

# 데이터프레임 생성
df = pd.DataFrame({
    "keyword": news_keywords,
    "title": news_titles,
    "label": news_labels
})

# CSV 저장
csv_filename = "stock_news_labeled_auto.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"\n✅ 총 {len(news_titles)}개 뉴스 수집 및 라벨링 완료!")
print(f"✅ '{csv_filename}' 파일로 저장 완료!")


'주가+상승' 뉴스 검색 중...
'주가+급등' 뉴스 검색 중...
'주가+하락' 뉴스 검색 중...
'주가+폭락' 뉴스 검색 중...

✅ 총 800개 뉴스 수집 및 라벨링 완료!
✅ 'stock_news_labeled_auto.csv' 파일로 저장 완료!


In [23]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# 검색할 키워드와 목표 수량 설정 + 라벨 정의
keywords = {
    "주가+상승": 1,
    "주가+급등": 1,
    "주가+하락": 0,
    "주가+폭락": 0
}
target_per_keyword = 200

# ✅ 띄어쓰기 없이 붙은 키워드들을 정리하기 위한 리스트
raw_keywords = ['주가상승', '주가급등', '주가하락', '주가폭락']

# 결과 저장 리스트
news_titles = []
news_keywords = []
news_labels = []

# 키워드별 수집
for keyword, label in keywords.items():
    print(f"'{keyword}' 뉴스 검색 중...")

    page = 1
    collected = 0

    while collected < target_per_keyword:
        url = f"https://search.daum.net/search?nil_suggest=btn&w=news&DA=SBC&cluster=y&q={keyword}&p={page}"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # 뉴스 제목 추출
        titles = soup.select('div.c-item-content strong.tit-g.clamp-g a')

        if not titles:
            print(f"❗ '{keyword}' 관련 뉴스가 더 이상 없습니다.")
            break

        for tag in titles:
            title = tag.get_text(strip=True)

            # ✅ 여기서 키워드 앞뒤로 띄어쓰기 무조건 삽입
            for raw_kw in raw_keywords:
                if raw_kw in title:
                    title = title.replace(raw_kw, f" {raw_kw} ")

            title = title.replace("'", "").replace('"', '')  # 따옴표 제거
            title = ' '.join(title.split())  # 중복 띄어쓰기 정리

            if title not in news_titles and title != '':
                news_titles.append(title)
                news_keywords.append(keyword)
                news_labels.append(label)
                collected += 1

            if collected >= target_per_keyword:
                break

        page += 1
        time.sleep(0.5)

# 데이터프레임 생성
df = pd.DataFrame({
    "keyword": news_keywords,
    "title": news_titles,
    "label": news_labels
})

# CSV 저장
csv_filename = "stock_news_labeled_auto.csv"
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

print(f"\n✅ 총 {len(news_titles)}개 뉴스 수집 및 라벨링 완료!")
print(f"✅ '{csv_filename}' 파일로 저장 완료!")


'주가+상승' 뉴스 검색 중...
'주가+급등' 뉴스 검색 중...
'주가+하락' 뉴스 검색 중...
'주가+폭락' 뉴스 검색 중...

✅ 총 800개 뉴스 수집 및 라벨링 완료!
✅ 'stock_news_labeled_auto.csv' 파일로 저장 완료!


In [24]:
# 데이터 증강

import pandas as pd
import random
from random import shuffle, sample

# 데이터 불러오기
df = pd.read_csv('stock_news_labeled_auto.csv', encoding='utf-8-sig')

# EDA 증강기 세팅
# 간단한 synonym dictionary
synonym_dict = {
    '상승': ['오름', '증가', '확대'],
    '하락': ['감소', '하향', '축소'],
    '호재': ['긍정', '좋은 소식'],
    '악재': ['부정', '나쁜 소식'],
    '주가': ['주식가격', '주식값'],
    '급등': ['급상승', '폭등'],
    '급락': ['급하락', '폭락']
}

def synonym_replacement(text):
    """동의어로 단어 교체"""
    words = text.split()
    new_words = words.copy()
    for i, word in enumerate(words):
        if word in synonym_dict:
            candidates = synonym_dict[word]
            new_words[i] = random.choice(candidates)
    return ' '.join(new_words)

def random_deletion(text, p=0.2):
    """단어 랜덤 삭제 (확률 p)"""
    words = text.split()
    if len(words) == 1:
        return text
    new_words = [word for word in words if random.uniform(0,1) > p]
    if len(new_words) == 0:
        return random.choice(words)
    else:
        return ' '.join(new_words)

def random_swap(text, n=1):
    """단어 n번 랜덤 위치 교환"""
    words = text.split()
    length = len(words)
    for _ in range(n):
        if length < 2:
            continue
        idx1, idx2 = random.sample(range(length), 2)
        words[idx1], words[idx2] = words[idx2], words[idx1]
    return ' '.join(words)

def random_insertion(text):
    """동의어를 랜덤 위치에 삽입"""
    words = text.split()
    new_words = words.copy()
    for _ in range(1):
        word_candidates = [word for word in words if word in synonym_dict]
        if not word_candidates:
            continue
        word_to_insert = random.choice(synonym_dict[random.choice(word_candidates)])
        insert_idx = random.randint(0, len(new_words))
        new_words.insert(insert_idx, word_to_insert)
    return ' '.join(new_words)

# 증강
augmented_texts = []
augmented_labels = []

for text, label in zip(df['title'], df['label']):
    for _ in range(2):  # 문장당 2개씩 증강
        operation = random.choice(['synonym', 'deletion', 'swap', 'insertion'])
        if operation == 'synonym':
            new_text = synonym_replacement(text)
        elif operation == 'deletion':
            new_text = random_deletion(text)
        elif operation == 'swap':
            new_text = random_swap(text)
        elif operation == 'insertion':
            new_text = random_insertion(text)
        else:
            new_text = text  # 원본유지

        augmented_texts.append(new_text)
        augmented_labels.append(label)

# 증강된 데이터프레임 만들기
aug_df = pd.DataFrame({
    'title': augmented_texts,
    'label': augmented_labels
})

# 원본 데이터 + 증강 데이터 합치기
combined_df = pd.concat([df[['title', 'label']], aug_df], ignore_index=True)

print(f"중복 제거 전 데이터 수: {len(combined_df)}개")

# 중복 제거
combined_df = combined_df.drop_duplicates(subset=['title'])

print(f"중복 제거 후 데이터 수: {len(combined_df)}개")

# 최종 저장
combined_df.to_csv('final_augmented_dataset.csv', encoding='utf-8-sig', index=False)
print("최종 CSV 저장 완료: final_augmented_dataset.csv")


중복 제거 전 데이터 수: 2400개
중복 제거 후 데이터 수: 1546개
최종 CSV 저장 완료: final_augmented_dataset.csv
